## Monte Carlo Integration

There are many integration techniques that can be used to compute integrals. But the vast majority of integrals do not have analytic answers! So we must resort to numerical methods of computing integrals.

When it comes to the numerical computation of integrals, there are several options:
- quadrature (eg: midpoint, trapezoidal, Simpson's rule)
- adaptive methods
- Newton-Cotes formulae.

But these options suffer from the curse of dimensionality. As the number of dimensions of the integral increases, you need in general, an exponential amount of function evaluations to compute the integral numerically.

So we consider Monte Carlo integration, which approximates the following integral as
$$\int_{[0, 1]^d} f(x)dx \approx \frac{1}{N} \sum_{i = 1}^{N} f(x_i)$$
where the domain of consideration is a d-dimensional hypercube $[0, 1]^d$, $x_1, x_2,..., x_N \sim U([0, 1]^d)$ are points uniformly sampled from the d-dimensional hypercube, and $N$ is the number of points that we sample.


One of the classic examples of Monte Carlo integration is the use of Monte Carlo integration to approximate $\pi$.

Consider the function 
$$f(x, y) =  \left\{
\begin{array}{ll}
      1 & x^2 + y^2 <= 1 \\
      0 & else \\
\end{array} 
\right.$$
and the domain $\Omega = [-1, 1] \times [-1, 1]$. 

We note that the integral
$$I = \int_{\Omega} f(x, y)dxdy = \pi $$
and thus by randomly (but crucially, uniformly) sampling $N$ points on $\Omega$, we can approximate $\pi$ by computing the raio between the number of points within the circle and the total number of sampled points. More precisely, this ratio approximates the area ratio of the circle and the rectangle

In mathematical terms,
$$I_{N} = \frac{1}{N} \sum_{i = 1}^{N} f(x_{i}, y_{i}) = \frac{\pi}{4} $$

In fact, this method can be generalized to estimate the area ratio of arbitrary shapes to a rectangle that inscribes the shape.


In [24]:
from numpy import random
import numpy as np
import math

n_batches = 10
n_samples = 100000
count = 0
radius = 1

for batch in range(n_batches):

    for i in range(n_samples):
        x_i = random.uniform(-1, 1)
        y_i = random.uniform(-1, 1)

        if x_i**2 + y_i**2 <= radius:
            count += 1

print(4*(1/(n_batches*n_samples))*count)
print(math.pi)

3.1422719999999997
3.141592653589793


Now we try the integration of some arbitrary functions. We consider the one-dimensional integral
$$ I_1 = \int_{0}^{2} cosh(x) dx$$


In [38]:
n_samples = 20000000
a, b = (0, 2)
rand_x = random.uniform(a, b, n_samples)
y = np.cosh(rand_x)
integral_approx = ((b - a)/n_samples)*y.sum()
print(integral_approx)


3.626620066769482
